In [3]:
# با استفاده از سایکیت لرن
import finpy_tse as fpy
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd

# بدون نیاز به نامپای و با پانداز پروژه انجام میشود

# جمع آوری داده 
i = [0]
historical_data = []
industries = ("خودرو" ,
              "فلزات اساسی", 
              "بانک",
              "دارویی"
             )
for industry in industries:
    industry = fpy.Get_SectorIndex_History(
        sector= industry,
        start_date='1402-01-01',
        end_date='1402-10-10',
        ignore_date=False,
        just_adj_close=True,
        show_weekday=False,
        double_date=False)
    historical_data.append(industry)
    i += i

df = pd.concat(historical_data, axis=1)
df.columns = industries

#  دیتای روز قبل و دو روز قبل برای هر صنعت استخراج و هر صنعت یک ستون دیتافریم جدید
for industry in industries:
    df[f'Previous_{industry}_Return'] = df[industry].shift(1)
    df[f'Second_Previous_{industry}_Return'] = df[industry].shift(2)

df.dropna(inplace=True)

print(df)

features = [f'Previous_{industry}_Return' for industry in industries] + [f'Second_Previous_{industry}_Return' for industry in industries]
X = df[features]  # Features
Y = df[[industry for industry in industries]]  # Targets

# تنظیم مدل 
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

# الگوریتم مدل جنگل تصادفی
model = RandomForestRegressor()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

next_day_prediction = model.predict([X.iloc[-1]])

predicted_returns = pd.Series(next_day_prediction[0], index=industries)
top_industry = predicted_returns.idxmax()

print(f"Predicted returns for the next day: {predicted_returns}")
print(f"Industry with the highest predicted profitability for the next day: {top_industry}")


               خودرو  فلزات اساسی    بانک    دارویی  Previous_خودرو_Return  \
J-Date                                                                       
1402-01-07  326118.0    1591350.0  7521.7  157763.0               322343.0   
1402-01-08  332138.0    1587240.0  7731.7  160537.0               326118.0   
1402-01-09  342696.0    1593310.0  7757.9  162924.0               332138.0   
1402-01-14  357558.0    1585160.0  7921.8  164359.0               342696.0   
1402-01-15  359554.0    1605290.0  8007.3  167835.0               357558.0   
...              ...          ...     ...       ...                    ...   
1402-10-04  320315.0    1683900.0  9322.5  208426.0               321546.0   
1402-10-05  315657.0    1666420.0  9278.7  208032.0               320315.0   
1402-10-06  316439.0    1665760.0  9270.5  208440.0               315657.0   
1402-10-09  318675.0    1671520.0  9316.7  209233.0               316439.0   
1402-10-10  318568.0    1661940.0  9289.4  208982.0             

C:\Users\Emad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
